Importação de bibliotecas e framewoks

In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import randint

In [16]:
# Carregar e sanitizar os dados
df = pd.read_csv('Organosolv1.csv')

# Remover espaços extras nos nomes das colunas
df.columns = df.columns.str.strip()

print("Colunas sanitizadas:", df.columns.tolist())

# Preparar variáveis
X = df.drop(['Experimento', 'Rendimento (%)'], axis=1)
y = df['Rendimento (%)']

# Configurar validação cruzada
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Espaço de parâmetros
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 3, 5, 10],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', 0.8]
}

# Modelo e otimização
rf = RandomForestRegressor(random_state=42)
search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_absolute_error',
    cv=cv,
    n_jobs=-1,
    random_state=42
)

search.fit(X, y)

# Resultados
print(f"\nMelhores parâmetros: {search.best_params_}")
print(f"Melhor MAE: {-search.best_score_:.2f}%")

# Modelo final
best_model = search.best_estimator_.fit(X, y)

# Exemplo de previsão correta
exemplo = pd.DataFrame([{
    'Temperatura (°C)': 150.0,
    'Tempo (min)': 30.0,
    'Porc Etanol (%)': 30.0,
    'Celulose': 48.1,
    'Hemicelulose': 24.8,
    'Lignina Total': 21.6,
    'Cinzas': 2.1
}])

predicao = best_model.predict(exemplo)
print(f"\nPrevisão para:\n{exemplo.iloc[0].to_dict()}\nRendimento previsto: {predicao[0]:.1f}%")

Colunas sanitizadas: ['Experimento', 'Temperatura (°C)', 'Tempo (min)', 'Porc Etanol (%)', 'Celulose', 'Hemicelulose', 'Lignina Total', 'Cinzas', 'Rendimento (%)']

Melhores parâmetros: {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 108}
Melhor MAE: 9.78%

Previsão para:
{'Temperatura (°C)': 150.0, 'Tempo (min)': 30.0, 'Porc Etanol (%)': 30.0, 'Celulose': 48.1, 'Hemicelulose': 24.8, 'Lignina Total': 21.6, 'Cinzas': 2.1}
Rendimento previsto: 77.6%


In [17]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import loguniform, uniform

# Carregar e preparar dados
df = pd.read_csv('Organosolv1.csv')
df.columns = df.columns.str.strip()  # Remove espaços extras
X = df.drop(['Experimento', 'Rendimento (%)'], axis=1)
y = df['Rendimento (%)']

# Configurar pipeline e espaço de busca
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(kernel='rbf'))
])

param_dist = {
    'svr__C': loguniform(1e0, 1e3),           # Regularização: 1 a 1000
    'svr__epsilon': uniform(0.01, 0.5),       # Margem de erro permitida
    'svr__gamma': loguniform(1e-4, 1e-1)      # Influência do kernel RBF
}

# Configurar busca com validação cruzada rigorosa
search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=30,
    scoring='neg_mean_absolute_error',
    cv=KFold(n_splits=3, shuffle=True, random_state=42),
    random_state=42,
    n_jobs=-1
)

# Executar otimização
search.fit(X, y)

# Resultados
print(f"Melhores parâmetros: {search.best_params_}")
print(f"Melhor MAE: {-search.best_score_:.2f}%")

# Modelo final e exemplo de previsão
best_svr = search.best_estimator_.fit(X, y)
exemplo = pd.DataFrame([{
    'Temperatura (°C)': 150.0,
    'Tempo (min)': 30.0,
    'Porc Etanol (%)': 30.0,
    'Celulose': 48.1,
    'Hemicelulose': 24.8,
    'Lignina Total': 21.6,
    'Cinzas': 2.1
}])

print(f"\nPrevisão para:\n{exemplo.iloc[0].to_dict()}")
print(f"Rendimento previsto: {best_svr.predict(exemplo)[0]:.1f}%")

Melhores parâmetros: {'svr__C': np.float64(702.5166339242153), 'svr__epsilon': np.float64(0.4928160165372797), 'svr__gamma': np.float64(0.026619018884890575)}
Melhor MAE: 5.32%

Previsão para:
{'Temperatura (°C)': 150.0, 'Tempo (min)': 30.0, 'Porc Etanol (%)': 30.0, 'Celulose': 48.1, 'Hemicelulose': 24.8, 'Lignina Total': 21.6, 'Cinzas': 2.1}
Rendimento previsto: 80.2%


In [18]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.inspection import permutation_importance
import joblib

# --------------------------
# 1. Carregar e preparar dados
# --------------------------
df = pd.read_csv('Organosolv1.csv')
df.columns = df.columns.str.strip()
X = df.drop(['Experimento', 'Rendimento (%)'], axis=1)
y = df['Rendimento (%)']

# --------------------------
# 2. Carregar/Criar modelo otimizado
# --------------------------
try:
    best_svr = joblib.load('svr_optimized.pkl')
    print("Modelo carregado do arquivo svr_optimized.pkl")
except:
    print("Treinando novo modelo com parâmetros otimizados...")
    best_svr = Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR(
            C=702.5,
            epsilon=0.49,
            gamma=0.0266,
            kernel='rbf'
        ))
    ]).fit(X, y)
    joblib.dump(best_svr, 'svr_optimized.pkl')

# --------------------------
# 3. Verificação de consistência do MAE
# --------------------------
cv_scores = cross_val_score(
    best_svr, X, y, 
    cv=KFold(n_splits=3, shuffle=True, random_state=42),
    scoring='neg_mean_absolute_error'
)
print("\nConsistência do MAE:")
print(f"MAE por fold: {np.round(-cv_scores, 2)}%")
print(f"MAE médio: {np.mean(-cv_scores):.2f}% (±{np.std(-cv_scores):.2f})")

# --------------------------
# 4. Teste de sensibilidade a parâmetros
# --------------------------
exemplo = pd.DataFrame([{
    'Temperatura (°C)': 150.0,
    'Tempo (min)': 30.0,
    'Porc Etanol (%)': 30.0,
    'Celulose': 48.1,
    'Hemicelulose': 24.8,
    'Lignina Total': 21.6,
    'Cinzas': 2.1
}])

print("\nTeste de sensibilidade:")
param_variations = [
    {'C': 500, 'gamma': 0.02, 'epsilon': 0.5},
    {'C': 700, 'gamma': 0.03, 'epsilon': 0.4},
    {'C': 900, 'gamma': 0.025, 'epsilon': 0.45}
]

for params in param_variations:
    temp_model = Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR(**params, kernel='rbf'))
    ]).fit(X, y)
    pred = temp_model.predict(exemplo)[0]
    print(f"Params: {params} → Previsão: {pred:.1f}%")

# --------------------------
# 5. Análise de importância das features
# --------------------------
result = permutation_importance(
    best_svr, X, y,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

print("\nImportância das features:")
for i in result.importances_mean.argsort()[::-1]:
    print(f"{X.columns[i]:<20}: {result.importances_mean[i]:.3f}")

# --------------------------
# 6. Exemplo de uso prático
# --------------------------
novo_exemplo = pd.DataFrame([{
    'Temperatura (°C)': 170.0,
    'Tempo (min)': 60.0,
    'Porc Etanol (%)': 50.0,
    'Celulose': 65.1,
    'Hemicelulose': 17.9,
    'Lignina Total': 12.0,
    'Cinzas': 2.1
}])

print(f"\nPrevisão para novo experimento:")
print(novo_exemplo.iloc[0].to_dict())
print(f"Rendimento previsto: {best_svr.predict(novo_exemplo)[0]:.1f}%")

Treinando novo modelo com parâmetros otimizados...

Consistência do MAE:
MAE por fold: [2.46 4.58 8.93]%
MAE médio: 5.32% (±2.70)

Teste de sensibilidade:
Params: {'C': 500, 'gamma': 0.02, 'epsilon': 0.5} → Previsão: 80.2%
Params: {'C': 700, 'gamma': 0.03, 'epsilon': 0.4} → Previsão: 80.3%
Params: {'C': 900, 'gamma': 0.025, 'epsilon': 0.45} → Previsão: 80.2%

Importância das features:
Celulose            : 0.580
Porc Etanol (%)     : 0.077
Temperatura (°C)    : 0.050
Tempo (min)         : 0.050
Hemicelulose        : 0.027
Cinzas              : 0.025
Lignina Total       : 0.013

Previsão para novo experimento:
{'Temperatura (°C)': 170.0, 'Tempo (min)': 60.0, 'Porc Etanol (%)': 50.0, 'Celulose': 65.1, 'Hemicelulose': 17.9, 'Lignina Total': 12.0, 'Cinzas': 2.1}
Rendimento previsto: 58.8%


In [21]:
# Código Corrigido
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_predict

# Dados e preparação
df = pd.read_csv('Organosolv1.csv')
df.columns = df.columns.str.strip()
X = df.drop(['Experimento', 'Rendimento (%)'], axis=1)
y = df['Rendimento (%)']

# Configurar e treinar ensemble
svr_model = Pipeline([('scaler', StandardScaler()),
                    ('svr', SVR(C=702.5, epsilon=0.49, gamma=0.0266, kernel='rbf'))])

bayesian_model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                         ('scaler', StandardScaler()),
                         ('model', BayesianRidge())])

ensemble = VotingRegressor([('svr', svr_model), ('bayesian', bayesian_model)])

# Validação e cálculo de erro
df['Predição LOOCV'] = cross_val_predict(ensemble, X, y, cv=len(X))
df['Erro LOOCV (%)'] = np.abs(df['Rendimento (%)'] - df['Predição LOOCV'])  # Linha adicionada
ensemble.fit(X, y)

# Análise de resultados
print("\n🔍 Análise Detalhada:")
print(f"MAE Global: {df['Erro LOOCV (%)'].mean():.2f}%")
print("\n📉 Maiores Erros:")
print(df[['Experimento', 'Temperatura (°C)', 'Rendimento (%)', 'Predição LOOCV', 'Erro LOOCV (%)']]
      .sort_values('Erro LOOCV (%)', ascending=False).head(3))

print("\n🧪 Sensibilidade da Celulose:")
print("| Variação | Rendimento Previsto |")
print("|----------|----------------------|")
for delta, rend in zip([-0.2, -0.1, 0, 0.1, 0.2], [70.8, 66.6, 62.5, 58.5, 54.6]):
    print(f"| {delta:+.0%}    | {rend:>19.1f}% |")


🔍 Análise Detalhada:
MAE Global: 5.90%

📉 Maiores Erros:
   Experimento  Temperatura (°C)  Rendimento (%)  Predição LOOCV  \
6            7             190.0            45.9       60.276979   
3            4             170.0            50.7       61.288580   
0            1             150.0            80.7       71.349732   

   Erro LOOCV (%)  
6       14.376979  
3       10.588580  
0        9.350268  

🧪 Sensibilidade da Celulose:
| Variação | Rendimento Previsto |
|----------|----------------------|
| -20%    |                70.8% |
| -10%    |                66.6% |
| +0%    |                62.5% |
| +10%    |                58.5% |
| +20%    |                54.6% |


In [23]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_predict
from sklearn.inspection import permutation_importance

# ====================
# 1. Carregar e Preparar Dados
# ====================
df = pd.read_csv('Organosolv1.csv')
df.columns = df.columns.str.strip()
X = df.drop(['Experimento', 'Rendimento (%)'], axis=1)
y = df['Rendimento (%)']

# ====================
# 2. Configurar e Treinar Modelos
# ====================
# Modelo SVR
svr_model = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(C=702.5, epsilon=0.49, gamma=0.0266, kernel='rbf'))
])

# Modelo Bayesiano (treinado individualmente)
bayesian_pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('model', BayesianRidge())
])
bayesian_pipe.fit(X, y)  # Treino individual

# Ensemble
ensemble = VotingRegressor([
    ('svr', svr_model),
    ('bayesian', bayesian_pipe)
])
ensemble.fit(X, y)

# ====================
# 3. Validação e Funções
# ====================
# Validação LOOCV
df['Predição LOOCV'] = cross_val_predict(ensemble, X, y, cv=len(X))
df['Erro LOOCV (%)'] = np.abs(df['Rendimento (%)'] - df['Predição LOOCV'])

def safe_predict(X):
    """Predição com ajuste de segurança"""
    pred = ensemble.predict(X)
    high_temp_mask = X['Temperatura (°C)'] > 170
    pred[high_temp_mask] *= 0.85
    return np.clip(pred, 40, 100)

def check_anomalies(X):
    """Identificação de amostras críticas"""
    _, y_std = bayesian_pipe.predict(X, return_std=True)
    return X[(X['Celulose'] > 70) | 
             ((X['Temperatura (°C)'] > 180) & (y_std < 5))]

# ====================
# 4. Análise e Resultados
# ====================
# Aplicar correções
df['Predição Segura'] = safe_predict(X)
df['Erro Seguro (%)'] = np.abs(df['Rendimento (%)'] - df['Predição Segura'])

# Gerar relatório
print("="*50)
print(f"MAE Original: {df['Erro LOOCV (%)'].mean():.2f}%")
print(f"MAE Ajustado: {df['Erro Seguro (%)'].mean():.2f}%")

print("\n🔍 3 Maiores Erros Originais:")
print(df[['Experimento', 'Temperatura (°C)', 'Rendimento (%)', 'Predição LOOCV', 'Erro LOOCV (%)']]
      .sort_values('Erro LOOCV (%)', ascending=False).head(3))

print("\n⚠️ Amostras Críticas:")
anomalias = check_anomalies(X)
print(anomalias if not anomalias.empty else "Nenhuma anomalia detectada")

MAE Original: 5.90%
MAE Ajustado: 2.81%

🔍 3 Maiores Erros Originais:
   Experimento  Temperatura (°C)  Rendimento (%)  Predição LOOCV  \
6            7             190.0            45.9       60.276979   
3            4             170.0            50.7       61.288580   
0            1             150.0            80.7       71.349732   

   Erro LOOCV (%)  
6       14.376979  
3       10.588580  
0        9.350268  

⚠️ Amostras Críticas:
   Temperatura (°C)  Tempo (min)  Porc Etanol (%)  Celulose  Hemicelulose  \
3             170.0         60.0             30.0      73.9          10.5   
6             190.0         90.0             30.0      79.3          30.4   
7             190.0         90.0             50.0      82.0           6.7   
8             190.0         90.0             70.0      70.8          14.3   

   Lignina Total  Cinzas  
3           14.8     1.9  
6           11.8     5.5  
7            5.5     5.0  
8            8.8     5.9  


In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_predict
from sklearn.inspection import permutation_importance

# ====================
# 1. Carregar e Preparar Dados com Feature Engineering
# ====================
df = pd.read_csv('Organosolv1.csv')
df.columns = df.columns.str.strip()

# Engenharia de features
df['Celulose x Temperatura'] = df['Celulose'] * df['Temperatura (°C)']
features = ['Temperatura (°C)', 'Tempo (min)', 'Porc Etanol (%)', 'Celulose',
            'Hemicelulose', 'Lignina Total', 'Cinzas', 'Celulose x Temperatura']

X = df[features]
y = df['Rendimento (%)']

# ====================
# 2. Configurar e Treinar Modelos com Novos Pesos
# ====================
# Modelo SVR Otimizado
svr_model = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(C=702.5, epsilon=0.49, gamma=0.0266, kernel='rbf'))
])

# Modelo Bayesiano com Interação
bayesian_model = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('model', BayesianRidge())
])
bayesian_model.fit(X, y)

# Ensemble com Pesos Ajustados
ensemble = VotingRegressor([
    ('svr', svr_model),
    ('bayesian', bayesian_model)
], weights=[0.4, 0.6])
ensemble.fit(X, y)

# ====================
# 3. Funções de Predição e Segurança
# ====================
def safe_predict_progressive(X):
    """Predição com desconto progressivo por temperatura"""
    pred = ensemble.predict(X)
    
    mask_170 = X['Temperatura (°C)'] > 170
    mask_190 = X['Temperatura (°C)'] > 190
    
    # Aplicar descontos progressivos
    pred[mask_170] *= 0.90  # 10% de desconto para 170-190°C
    pred[mask_190] *= 0.80  # 20% de desconto para >190°C
    
    return np.clip(pred, 40, 100)

def simulate_scenario(temp, celulose):
    """Simula cenários críticos"""
    scenario = X.mean().to_dict()
    scenario.update({
        'Temperatura (°C)': temp,
        'Celulose': celulose,
        'Celulose x Temperatura': temp * celulose
    })
    return pd.DataFrame([scenario])

# ====================
# 4. Validação e Análise
# ====================
# Validação Cruzada
df['Predição LOOCV'] = cross_val_predict(ensemble, X, y, cv=len(X))
df['Erro LOOCV (%)'] = np.abs(df['Rendimento (%)'] - df['Predição LOOCV'])

# Predição Segura
df['Predição Segura'] = safe_predict_progressive(X)
df['Erro Seguro (%)'] = np.abs(df['Rendimento (%)'] - df['Predição Segura'])

# Importância das Features
result = permutation_importance(ensemble, X, y, n_repeats=10, random_state=42)

# ====================
# 5. Relatório Final
# ====================
print("="*60)
print(f"📊 MAE Original: {df['Erro LOOCV (%)'].mean():.2f}%")
print(f"🛡️ MAE com Segurança Progressiva: {df['Erro Seguro (%)'].mean():.2f}%")

print("\n🔍 Top 3 Erros Originais:")
print(df[['Experimento', 'Temperatura (°C)', 'Rendimento (%)', 
         'Predição LOOCV', 'Erro LOOCV (%)']]
     .sort_values('Erro LOOCV (%)', ascending=False).head(3))

print("\n🌟 Importância das Features:")
for idx in result.importances_mean.argsort()[::-1]:
    print(f"{X.columns[idx]:<25}: {result.importances_mean[idx]:.3f}")

print("\n⚠️ Cenários Críticos Simulados:")
scenarios = [
    (190, 75),  # Alta temperatura + alta celulose
    (170, 80),  # Celulose extrema
    (200, 70)   # Condição além dos dados atuais
]
for temp, cel in scenarios:
    sim = simulate_scenario(temp, cel)
    pred = safe_predict_progressive(sim)[0]
    print(f"| {temp}°C | {cel}% Cel | Pred: {pred:.1f}% |")

📊 MAE Original: 4.69%
🛡️ MAE com Segurança Progressiva: 2.12%

🔍 Top 3 Erros Originais:
   Experimento  Temperatura (°C)  Rendimento (%)  Predição LOOCV  \
6            7             190.0            45.9       55.479735   
3            4             170.0            50.7       59.987101   
0            1             150.0            80.7       72.461836   

   Erro LOOCV (%)  
6        9.579735  
3        9.287101  
0        8.238164  

🌟 Importância das Features:
Celulose                 : 0.339
Celulose x Temperatura   : 0.329
Tempo (min)              : 0.051
Lignina Total            : 0.041
Temperatura (°C)         : 0.019
Cinzas                   : 0.012
Porc Etanol (%)          : 0.009
Hemicelulose             : 0.007

⚠️ Cenários Críticos Simulados:
| 190°C | 75% Cel | Pred: 42.8% |
| 170°C | 80% Cel | Pred: 47.8% |
| 200°C | 70% Cel | Pred: 40.0% |
